In [10]:
import pandas as pd
debian_mailing_list = pd.read_csv("./debian-user-filtered.dat", header = None)

processed_dml = []
for items in debian_mailing_list[0]:
    link = [int(s) for s in items.split(" ")]
    processed_dml.append(link)
    
threads_dict = {}
for link in processed_dml:
    thread_id = link[4]
    if thread_id not in threads_dict:
        threads_dict.update({thread_id : [link[0:3]]})
    else:
        threads_dict[thread_id].append(link[0:3])

In [23]:
test_data = []
partition_data = []
i = 1
for key in threads_dict:
    if i < 200:
        test_data.append(threads_dict[key])
    elif i < 400:
        partition_data.append(threads_dict[key])
    else:
        break
        
    i += 1

In [17]:
def time_stamp(link):
    return link[0]

def return_all_statistic(partition):
    num_of_links = []
    num_of_nodes = []
    duration = []
    variance = []
    
    for thread in partition:
        thread = sorted(thread, key = time_stamp)
        duration.append((thread[-1][0] - thread[0][0])/day)
        
        num_of_links.append(len(thread))
        
        thread_node = []
        for link in thread:
            for node in link[1:3]:
                if node not in thread_node:
                    thread_node.append(node)
        num_of_nodes.append(len(thread_node))                                                                                                                          
        
        if len(thread) < 2:
            variance.append(0)
            continue
        
        intercontact_time = []
        for i in range(len(thread)-1):
            intercontact_time.append((thread[i+1][0] - thread[i][0])/day)
            
        expectation = (thread[-1][0] - thread[0][0]) / (day * len(intercontact_time)) 
        
        var = 0
        for t in intercontact_time:
            var += (t - expectation)**2
        var = var / len(intercontact_time)
        variance.append(var)
        
    return num_of_links, num_of_nodes, duration, variance

In [18]:
import numpy as np
import math
day = 24*60*60
def thread_score(num_of_links, num_of_nodes, duration, variance, alpha, beta, gamma):
    return (num_of_links ** gamma) / ( num_of_nodes  + alpha * variance + beta * duration)

thread_score = np.vectorize(thread_score)

def partition_value(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma):
    total_links = np.sum(list_num_links)
    return np.sum(thread_score(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma)) /total_links

def partial_alpha(num_of_links, num_of_nodes, duration, variance, alpha, beta, gamma):
    partial_alpha = - variance * (num_of_links ** gamma) / ((num_of_nodes + alpha * variance + beta * duration) ** 2)
    return partial_alpha
partial_alpha = np.vectorize(partial_alpha)

def partial_beta(num_of_links, num_of_nodes, duration, variance, alpha, beta, gamma):
    partial_beta = - duration * (num_of_links ** gamma) / ((num_of_nodes + alpha * variance + beta * duration) ** 2)
    return partial_beta
partial_beta = np.vectorize(partial_beta)

def partial_gamma(num_of_links, num_of_nodes, duration, variance, alpha, beta, gamma):
    partial_gamma = math.log(gamma) * (num_of_links ** gamma) / (num_of_nodes + alpha * variance + beta * duration)
    return partial_gamma
partial_gamma = np.vectorize(partial_gamma)

def grad_partition_value(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma):
    total_links = np.sum(list_num_links)
    p_alpha = partial_alpha(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma)
    p_beta =  partial_beta(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma)
    p_gamma = partial_gamma(list_num_links, list_num_node, list_duration, list_variance, alpha, beta, gamma)
    return np.sum(p_alpha)/total_links, np.sum(p_beta)/total_links, np.sum(p_gamma)/total_links

In [19]:
def finding_nemo(ground_truth_partition, alpha = 1, beta = 1, gamma = 1.5, learning_rate = 0.05, back_track = 0.5):
    big_thread = []
    for thread in ground_truth_partition:
        for link in thread:
            big_thread.append(link)
    whole_partition = [big_thread]
    
    whole_stat = return_all_statistic(whole_partition)
    gt_stat = return_all_statistic(ground_truth_partition)
    
    # first try to optimize f(E_g) to a value greater than 0.5
    a = alpha
    b = beta
    g = gamma
    lr = learning_rate
    value = partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], alpha, beta, gamma)
    while value < 0.55:
        grad = grad_partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], a, b, g)
        a_tmp = a + lr * grad[0]
        b_tmp = b + lr * grad[1]
        g_tmp = g + lr * grad[2]
        
        i = 0
        while not (a_tmp >= 0 and b_tmp >= 0 and g_tmp >=1):
            lr = learning_rate * back_track
            a_tmp = a + lr * grad[0]
            b_tmp = b + lr * grad[1]
            g_tmp = g + lr * grad[2]
            i += 1
            if i > 100:
                print('feu vkl')
                return
        a = a_tmp
        b = b_tmp
        g = g_tmp
        value = partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], a, b, g)
        
    whole_value = partition_value(whole_stat[0], whole_stat[1], whole_stat[2], whole_stat[3], a, b, g)
    prev_value = -100
    current_value = value - whole_value
    i = 0
    while not (current_value > 0 and current_value - prev_value < 0.1):
        i += 1
        whole_grad = grad_partition_value(whole_stat[0], whole_stat[1], whole_stat[2], whole_stat[3], a, b, g)
        grad = grad_partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], a, b, g)
        some_grad = np.array(grad) - np.array(whole_grad)
        a_tmp = a + lr * some_grad[0]
        b_tmp = b + lr * some_grad[1]
        g_tmp = g + lr * some_grad[2]
        
        j = 0
        while not (a_tmp >= 0 and b_tmp >= 0 and g_tmp >=1 and partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], a_tmp, b_tmp, g_tmp) > 0.6):
            lr = learning_rate * back_track
            a_tmp = a + lr * some_grad[0]
            b_tmp = b + lr * some_grad[1]
            g_tmp = g + lr * some_grad[2]
            j += 1
            if j > 100:
                print('fail vkl')
                return
        a = a_tmp
        b = b_tmp
        g = g_tmp
        
        
        prev_value = current_value
        whole_value = partition_value(whole_stat[0], whole_stat[1], whole_stat[2], whole_stat[3], a, b, g)
        partition_value(gt_stat[0], gt_stat[1], gt_stat[2], gt_stat[3], a, b, g)
        current_value = value - whole_value
        
        if i > 1000:
            break
    print(i)
    print(a, b, g)

In [15]:
import time
start_time = time.time()
finding_nemo(test_data, 0.01, 0.1, 1.7)
print(time.time() - start_time)

1
0.007888319254115064 0.09711750171800987 1.7064300003938953
0.003969669342041016


In [26]:
alpha = 0.00789
beta = 0.09712
gamma = 1.70643

need_partition_links = []
for thread in partition_data:
    for link in thread:
        need_partition_links.append(link)

In [126]:
import itertools 

def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

def union(lst1, lst2):
    lst3 = lst1
    for value in lst2:
        if value not in lst3:
            lst3.append(value)
    return lst3

def max_intersection_thread(cluster, ground_truth_partition):
    max_intersection = 0
    max_thread = None

    for thread in ground_truth_partition:
        intersect = intersection(cluster, thread)
        if max_intersection < len(intersect):
            max_intersection = len(intersect)
            max_thread = thread
    return (max_thread, max_intersection)

def partition_score(partition, ground_truth_partition):
    final_score = 0
    for cluster in partition:
        match_thread, intersect = max_intersection_thread(cluster, ground_truth_partition)
        cluster_score = intersect / len(union(cluster, match_thread))
        final_score += cluster_score
    return final_score / len(partition)


def findsubsets(s, n): 
    return list(itertools.combinations(s, n))


def greedy_partition(links, k = 2): #k is probing step
    #sort all link in time order
    links = sorted(links, key = time_stamp)
    
    #for each link, consider k link forward, choose the highest score among subset links of k
    #continue the search, until cannot add more link on that thread
    #remove all links of the thread for links, and start the search again for the first link
    
    partition = []
    while links: #stop criterion
        init_link = links[0]
        best_candidate = [init_link]
        max_score = 0.5
        link_set = links[1:k+1]
        for i in range(1, k+1):
            for subset in findsubsets(link_set, i):
                thread = [init_link]
                for thr in subset:
                    thread.append(list(thr))
                thread = sorted(thread, key = time_stamp)
                
                num_of_links = len(thread)

                nodes = []
                for link in thread:
                    for node in link[1:3]:
                        if node not in nodes:
                            nodes.append(node)
                num_of_nodes = len(nodes)
                intercontact = []
                for j in range(0, i):
                    intercontact.append((thread[j+1][0]-thread[j][0]) / day)
                    
                duration = (thread[-1][0] - thread[0][0]) /day
                expectation = duration / len(intercontact)
                variance = 0
                for t in intercontact:
                    variance += (t - expectation)**2
                variance = variance / len(intercontact)
                
                score = thread_score(num_of_links, num_of_nodes, duration, variance, alpha, beta, gamma) / num_of_links
                if score > max_score:
                    max_score = score
                    best_candidate = thread

        partition.append(best_candidate)
        for l in best_candidate:
            links.remove(l)
    return partition

In [140]:
a = greedy_partition(need_partition_links, 5)

In [141]:
stat = return_all_statistic(a)
partition_value(stat[0], stat[1], stat[2], stat[3], alpha, beta, gamma)

0.5515619426615703

In [142]:
partition_score(a, partition_data)

0.6116827882378122

In [116]:
thread_score(3, 4, 0.86551, 0.0733, alpha, beta, gamma)

array(1.59595481)

In [109]:
la = return_all_statistic(partition_data)

In [114]:
partition_value(la[0], la[1], la[2], la[3], alpha, beta, gamma)

0.5673644384741344

In [75]:
partition_data

[[[5716896, 116, 269]],
 [[5719234, 16, 28]],
 [[5726782, 239, 273]],
 [[5738172, 25, 277], [5770917, 116, 25], [5812952, 98, 116]],
 [[5754031, 58, 164]],
 [[5754149, 16, 278], [5757599, 278, 16]],
 [[5762534, 93, 226]],
 [[5763128, 93, 44]],
 [[5769139, 66, 28]],
 [[5794605, 11, 279]],
 [[5812861, 98, 44]],
 [[5826223, 16, 283]],
 [[5854133, 16, 282],
  [5854856, 83, 16],
  [5875692, 15, 282],
  [5884603, 281, 282],
  [5911340, 55, 282]],
 [[5862072, 25, 145]],
 [[5948541, 216, 171]],
 [[5958503, 37, 50]],
 [[6017182, 244, 283]],
 [[6022692, 25, 95], [6024947, 49, 95]],
 [[6035203, 233, 285], [6049212, 111, 285]],
 [[6036947, 83, 140], [6054060, 22, 140]],
 [[6051728, 119, 280]],
 [[6069598, 55, 286]],
 [[6158832, 25, 106], [6171012, 15, 106]],
 [[6198732, 63, 202]],
 [[6228832, 20, 95], [6235316, 32, 95], [6248832, 111, 20], [6254412, 15, 95]],
 [[6232793, 126, 197],
  [6235592, 23, 126],
  [6450241, 292, 23],
  [6491175, 296, 292]],
 [[6256480, 248, 294],
  [6302613, 116, 248],
  [

In [143]:
a

[[[5716896, 116, 269]],
 [[5719234, 16, 28], [5754149, 16, 278], [5757599, 278, 16]],
 [[5726782, 239, 273]],
 [[5738172, 25, 277], [5770917, 116, 25], [5812952, 98, 116]],
 [[5754031, 58, 164]],
 [[5762534, 93, 226], [5763128, 93, 44]],
 [[5769139, 66, 28]],
 [[5794605, 11, 279]],
 [[5812861, 98, 44],
  [5812952, 98, 116],
  [5738172, 25, 277],
  [5770917, 116, 25]],
 [[5826223, 16, 283],
  [5854133, 16, 282],
  [5854856, 83, 16],
  [5875692, 15, 282],
  [5884603, 281, 282],
  [5911340, 55, 282]],
 [[5862072, 25, 145]],
 [[5875692, 15, 282],
  [5884603, 281, 282],
  [5854133, 16, 282],
  [5854856, 83, 16],
  [5911340, 55, 282]],
 [[5911340, 55, 282],
  [5854133, 16, 282],
  [5854856, 83, 16],
  [5875692, 15, 282],
  [5884603, 281, 282]],
 [[5948541, 216, 171]],
 [[5958503, 37, 50]],
 [[6017182, 244, 283]],
 [[6022692, 25, 95], [6024947, 49, 95]],
 [[6035203, 233, 285], [6049212, 111, 285]],
 [[6036947, 83, 140], [6054060, 22, 140]],
 [[6051728, 119, 280]],
 [[6069598, 55, 286]],
 [[61

In [144]:
17/24

0.7083333333333334

In [90]:
stat[1][2]

5

In [39]:
from numpy import linalg as LA
w, v = LA.eig(np.array(
    [[2, -1, -1], 
     [-1, 1, 0],
     [-1, 0, 1]]))
w

array([ 3.00000000e+00, -2.22044605e-16,  1.00000000e+00])

In [105]:
w, v = LA.eig(np.array(
    [[1, -1/2, -1/2, -1/2, -1/2], 
     [-1/2, 1, 0, 0, 0],
     [-1/2, 0, 1, 0, 0],
     [-1/2, 0, 0, 1, 0],
     [-1/2, 0, 0, 0, 1]]))
w

array([2., 0., 1., 1., 1.])

In [106]:
w, v = LA.eig(np.array(
    [[1, -1/2, -1/2], 
     [-1/2, 1, -1/2],
     [-1/2, -1/2, 1]]))
w

array([ 1.50000000e+00, -5.55111512e-17,  1.50000000e+00])

In [118]:
w, v = LA.eig(np.array(
    [[1, -1/1.4142, -1/1.4142], 
     [-1/1.4142, 1, 0],
     [-1/1.4142, 0, 1]]))
w

array([ 2.00000959e+00, -9.59013795e-06,  1.00000000e+00])

In [117]:
1.4142**2

1.9999616399999998

In [ ]:
# import random
# def generate_partition(ground_truth_partition): #create a path from ground truth partition to root
#     current_length = len(ground_truth_partition)
#     prev_partition = ground_truth_partition
#     result = []
#     for i in range(len(ground_truth_partition)-1):
#         new_thread = prev_partition[0]
#         for link in prev_partition[1]:
#             new_thread.append(link)
            
#         new_partition = []
#         new_partition.append(new_thread)
#         for thread in prev_partition[2:]:
#             new_partition.append(thread)
        
        
#         prev_partition = new_partition
#         current_length -= 1
        
#         result.append(new_partition)
#     return result